# Update Critters Table

In [ ]:
from arcgis.gis import GIS
from arcgis import features
from arcgis.features import FeatureLayerCollection
import ipywidgets as widgets
import pandas as pd
import io
import warnings

def update_table(b):    
    with output:
        # Clear any previous output
        output.clear_output()
        
        # Initialize progress bar
        progress = widgets.IntProgress(value=0, min=0, max=4, step=1, description='Progress:')
        display(progress)
        
        # Check if file is uploaded
        if uploader.value:
            
            try:
                # Update progress
                progress.value = 1
                
                # Read in uploaded file as a csv
                uploaded_file = next(iter(uploader.value.values()))
                csv = io.BytesIO(uploaded_file['content'])

                # Read uploaded file into a Pandas DataFrame
                df = pd.read_csv(csv)
                
                # Update progress
                progress.value = 2

                 # Convert 'Mon_Date_Time' column to datetime
                df['Session_Date'] = pd.to_datetime(df['Mon_Date_Time'])
                
                # Group by 'site' and find rows with max 'session_date' for each 'site'
                df = df[df.groupby('WQMSite ID')['Session_Date'].transform('max') == df['Mon_Date_Time']]

                # Rename lat/lon
                df.rename(columns={"LongLat (Latitude)": "Longitude", "LongLat (Longitude)": "Latitude"}, inplace=True)
                
                # Drop null values (+ last 5 Salesforce lines)
                df.dropna(subset=['Longitude', 'Latitude'], inplace=True)

                # Upload the csv to files
                df.to_csv('critters.csv', index=False)
                
                # Update progress
                progress.value = 3
                  
                # Search for Critters table
                items = gis.content.search(query='title:Critters', item_type='Feature Service')
                critters_found = False
                for item in items:
                    if item.title == "Critters":
                        critters_found = True
                        
                        # Replace table item
                        collection = FeatureLayerCollection.fromitem(item)
                        csv_path = '/arcgis/critters.csv'
                        collection.manager.overwrite(csv_path)
                
                # IBI Scores table not found, create new table
                if not critters_found:        
                    item_properties = {
                        "title": "Critters",
                    }
                    n_item = gis.content.add(item_properties=item_properties, data='/arcgis/critters.csv')
                    publish_properties = {
                        "type:" : "csv",
                        "locationType" : "none",
                    }
                    p_item = item.publish(publish_parameters=publish_properties)
                
                # Clean up temporary file
                os.remove('/arcgis/critters.csv')
            
                # Update progress
                progress.value = 4

                print('Critters table updated.')
            
            except Exception as e:
                print(f"Error: {str(e)}")
                        
        else:
            print("Error: No file uploaded.")
            
# Suppress admin warning
warnings.filterwarnings("ignore", message="You are logged on as .* with an administrator role")
            
# GIS connection
gis = GIS("home")            
            
# Initialize widgets and button click event
output = widgets.Output()
uploader = widgets.FileUpload(accept='.csv', multiple=False)
button = widgets.Button(description='Update Table', tooltip='Click me')
button.on_click(update_table)

# Display widgets and button
print('\nUpload LimnoTech Session/Critter report (csv format).')
display(uploader)

print('\nClick the button to update the table.')
display(button)
display(output)